In [1]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold, KFold
from sklearn import linear_model, metrics, preprocessing
from math import sqrt, floor
from datetime import datetime, timedelta
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input, LSTM,Dense,RepeatVector, TimeDistributed , LeakyReLU
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, IsolationForest
from sklearn.metrics import r2_score, make_scorer, accuracy_score, mean_squared_error, mean_absolute_error
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
import tensorflow as tf
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
import pandas as pd
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)
import time
from sklearn.pipeline import make_pipeline, Pipeline
import math
from sklearn.decomposition import PCA
from numpy import array
from keras.utils.vis_utils import plot_model
import keras
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input,Conv2D, Conv1D, Conv1DTranspose,MaxPooling1D
import json
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from keras import optimizers, Sequential
from sklearn.ensemble import IsolationForest
from statistics import mean
from sklearn.metrics import f1_score
import pickle

2023-09-01 21:06:35.157651: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-09-01 21:06:35.157690: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import pickle
from sklearn.metrics import f1_score, accuracy_score,precision_score, recall_score
def getScore(df):
    # print(df)
    return [f1_score(y_true=df['ytrue'], y_pred=df['ypred_mse'])]
def getDf(channel, path, dayRange):
#     print(dayRange)
    ae_df, convae_df, lstmae_df, lstm_attnae_df, tcnae_df = pd.DataFrame(), pd.DataFrame(),pd.DataFrame(),pd.DataFrame(), pd.DataFrame()
    files = os.listdir(path)
#     print(files)
    if '.ipynb_checkpoints' in files:
        files.remove('.ipynb_checkpoints')
    for file in files:
        with open(f'{path}{file}', 'rb') as f:
                    if 'STL_ConvAE' in file:
                        df_dict = pickle.load(f)
#                         print(df_dict)
                        convae_df = df_dict[dayRange][channel]
                        convae_df = convae_df.dropna()
                    elif 'STL_LstmAE' in file:
                        df_dict = pickle.load(f)
                        lstmae_df = df_dict[dayRange][channel]
                        lstmae_df = lstmae_df.dropna()
                    elif 'STL_TcnAE' in file:
                        df_dict = pickle.load(f)
                        tcnae_df = df_dict[dayRange][channel]
                        tcnae_df = tcnae_df.dropna()
                    elif 'STL_LstmAttnAE' in file:
                        df_dict = pickle.load(f)
                        lstm_attnae_df = df_dict[dayRange][channel]
                        lstm_attnae_df = lstm_attnae_df.dropna()
                    elif 'STL_AE' in file:
                        df_dict = pickle.load(f)
                        ae_df = df_dict[dayRange][channel]
                        ae_df= ae_df.dropna()
    return ae_df, convae_df, lstmae_df, lstm_attnae_df, tcnae_df

In [3]:
def getDynamicThreshold(df, tail=100, w_y=0.3, w_z=0.7):
    train_mean = df['mse_scores'].mean()
    train_std = df['mse_scores'].std()
    df = df.tail(tail)
    tail_mean = df['mse_scores'].mean()
    tail_std = df['mse_scores'].std()
    threshold = w_y*(train_mean+train_std)+w_z*(tail_mean+tail_std)
    return threshold
def getthreshold(df_train, df_test):
    df_tr = df_train[['ytrue', 'mse_scores']]
    df_te = df_test[['ytrue', 'mse_scores']]
    thresholds=[]
    for i in range(df_test.shape[0]):
        df_tr = pd.concat([df_tr, df_te[i:i+1]], axis = 0)
        thresholds.append(getDynamicThreshold(df_tr,  w_y=0.4, w_z=0.6))
#         thresholds.append(getDynamicThreshold(df_tr))
    
    df_test['thresholds'] = thresholds
#     df_test['thresholds'] = getDynamicThreshold(df_tr,  w_y=0.5, w_z=0.5)
    return df_test

In [4]:
def getDataFrames(channel, itr, dayRange):
    train_path = f'datapath'
    test_path = f'datapath'
    ae_df_train, convae_df_train, lstmae_df_train, lstm_attnae_df_train, tcnae_df_train = getDf(channel, train_path, dayRange)
    ae_df_test, convae_df_test, lstmae_df_test, lstm_attnae_df_test,tcnae_df_test = getDf(channel, test_path, dayRange)
#     print(ae_df_test.shape, convae_df_test.shape, lstmae_df_test.shape, lstm_attnae_df_test.shape, tcnae_df_test.shape)
    ae_df  = getthreshold(ae_df_train, ae_df_test)
    convae_df = getthreshold(convae_df_train, convae_df_test)
    lstmae_df = getthreshold(lstmae_df_train, lstmae_df_test)
    lstm_attnae_df = getthreshold(lstm_attnae_df_train, lstm_attnae_df_test)
    
    tcnae_df = getthreshold(tcnae_df_train, tcnae_df_test)
   
    return  ae_df, convae_df, lstmae_df, lstm_attnae_df, tcnae_df
#     return ae_df_test, convae_df_test, lstmae_df_test, lstm_attnae_df_test,tcnae_df_test

In [5]:
def getDfLearn(metric, itr, dayRange):
    df_learn = pd.DataFrame()
    for i in range(19):
        channel = f'inv_{i}'
        ae_df, convae_df, lstmae_df, lstm_attnae_df, tcnae_df = getDataFrames(channel, itr, dayRange)
        
        df_learn = pd.concat([df_learn,pd.DataFrame({'file':channel,'ae':getScore(ae_df,metric,channel,'ae'),'convae':getScore(convae_df,metric,channel,'convae'),
            'tcnae':getScore(tcnae_df, metric, channel, 'tcnae'),'lstmae':getScore(lstmae_df,metric,channel,'lstmae'), 
                                    'lstm_attnae':getScore(lstm_attnae_df,metric,channel, 'lstm_attnae')})], axis=0)
    
    df_learn = df_learn.set_index(df_learn['file'])
    df_learn = df_learn.drop(columns=['file'])
    return df_learn

In [6]:

def getScore(df,metric, channel,name_dataframe ):
#     print(name_dataframe)
    if name_dataframe=='isofor' or name_dataframe=='mtl_isofor':
        df['ypred'] = df['ypred_mse']
    else:
        df['ypred'] = np.where(df['mse_scores']>df['thresholds'],1,0)
#         df['ypred'] = df['ypred_mse']
    if metric == 'f1':
        return [f1_score(y_true=df['ytrue'], y_pred=df['ypred'])]    
    if metric == 'acc':
        return [accuracy_score(y_true=df['ytrue'], y_pred=df['ypred'])]
    if metric == 'prec':
        return [precision_score(y_true=df['ytrue'], y_pred=df['ypred'])]
    if metric == 'rec':
        return [recall_score(y_true=df['ytrue'], y_pred=df['ypred'])]
df_metric = pd.DataFrame()
df_zsl_metric = pd.DataFrame()
df_source_metric = pd.DataFrame()
for metric in ['rec']:
    print(metric)
    df = pd.DataFrame()
    for daysRange in [14, 30, 90]:
        df_learn1 = getDfLearn(metric, 1, daysRange)

        df_learn2 = getDfLearn(metric, 2, daysRange)
        df_learn3 = getDfLearn(metric, 3, daysRange)

        df_learn = (df_learn1 + df_learn2 + df_learn3)/3

        df = pd.concat([df,df_learn.mean()], axis=1)
    df_metric = pd.concat([df_metric,df], axis=1)
    print(df_metric)
    
                             

rec
                    0         0         0
ae           0.748058  0.694276  0.692049
convae       0.750525  0.693784  0.701674
tcnae        0.778615  0.764932  0.724389
lstmae       0.733916  0.636777  0.557720
lstm_attnae  0.746884  0.699152  0.724796


In [25]:
df_metric.to_csv('./MaxThresholding.csv')

In [66]:
df_metric.to_csv('./dynamicThresholding0505.csv')

In [ ]:
df_metric.to_csv('./FloatingdynamicThresholding0406.csv')

In [39]:
df = pd.read_csv('/mnt/stud/home/vpulagura/experiment/OpenDatasets/SWaT/data/SWaTNormal.csv')
df['Timestamp'] = pd.to_datetime(df[' Timestamp'])
df = df.set_index(df['Timestamp'])
df = df.drop(columns=['Timestamp',' Timestamp'])
df['Normal/Attack'] = np.where(df['Normal/Attack'] == 'Normal',0,1)
df_down = df.resample('0.5T').mean()
df_down =  df_down.drop_duplicates()
df_down = df_down.dropna()
df_down.to_csv('/mnt/stud/home/vpulagura/experiment/OpenDatasets/SWaT/data/DownsampledNormal.csv')

In [2]:
df = pd.read_csv('/mnt/stud/home/vpulagura/experiment/OpenDatasets/SWaT/data/SWaTNormal.csv')
df.shape

(495000, 53)

In [4]:
df = pd.read_csv('/mnt/stud/home/vpulagura/experiment/OpenDatasets/SWaT/data/SWaTAttack.csv')
df[df['Normal/Attack'] == 'Normal'].shape

(395298, 53)

In [41]:
df = pd.read_csv('/mnt/stud/home/vpulagura/experiment/OpenDatasets/SWaT/data/SWaTAttack.csv')
df['Timestamp'] = pd.to_datetime(df[' Timestamp'])
df = df.set_index(df['Timestamp'])
df = df.drop(columns=['Timestamp',' Timestamp'])
df['Normal/Attack'] = np.where(df['Normal/Attack'] == 'Normal',0,1)
df_down = df.resample('0.5T').mean()
df_down =  df_down.drop_duplicates()
df_down = df_down.dropna()
df_down['Normal/Attack'] = np.where(df_down['Normal/Attack']>0,1,0)
df_down.to_csv('/mnt/stud/home/vpulagura/experiment/OpenDatasets/SWaT/data/DownsampledAttack.csv')

In [45]:
df= pd.read_csv('/mnt/stud/home/vpulagura/experiment/OpenDatasets/SWaT/data/DownsampledNormal.csv')
df

,Timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,MV201,P201,P202,P203,P204,P205,P206,DPIT301,FIT301,LIT301,MV301,MV302,MV303,MV304,P301,P302,AIT401,AIT402,FIT401,LIT401,P401,P402,P403,P404,UV401,AIT501,AIT502,AIT503,AIT504,FIT501,FIT502,FIT503,FIT504,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,Normal/Attack
0,2015-12-22 16:30:00,0.000000,124.096297,1.0,1.0,1.0,251.922600,8.311000,312.832623,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.560983,0.000256,138.001473,1.0,1.0,1.0,1.0,1.0,1.0,0.000,169.238700,0.000000,133.551660,1.0,1.0,1.0,1.0,1.0,7.445559,175.396040,260.702400,123.347827,0.001538,0.001409,0.001664,0.000000,1.0,1.0,9.100231,0.000000,3.348500,0.000256,1.0,1.0,1.0,0.0
1,2015-12-22 16:30:30,0.000000,124.220587,1.0,1.0,1.0,251.922600,8.309002,313.035133,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.560983,0.000256,137.997463,1.0,1.0,1.0,1.0,1.0,1.0,0.000,169.336927,0.000000,132.621120,1.0,1.0,1.0,1.0,1.0,7.444096,175.421673,260.702400,123.415783,0.001538,0.001409,0.001664,0.000000,1.0,1.0,9.058041,0.000000,3.348500,0.000256,1.0,1.0,1.0,0.0
2,2015-12-22 16:31:00,0.000000,124.029560,1.0,1.0,1.0,251.922600,8.309986,313.201733,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.560983,0.000256,137.650337,1.0,1.0,1.0,1.0,1.0,1.0,0.000,169.533480,0.000000,133.238917,1.0,1.0,1.0,1.0,1.0,7.444544,175.603710,260.702400,123.379893,0.001538,0.001409,0.001664,0.000000,1.0,1.0,9.068188,0.000000,3.348500,0.000256,1.0,1.0,1.0,0.0
3,2015-12-22 16:31:30,0.000032,123.787493,1.0,1.0,1.0,251.922600,8.309515,313.391437,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.562371,0.000256,137.475430,1.0,1.0,1.0,1.0,1.0,1.0,0.000,169.596697,0.000000,132.054613,1.0,1.0,1.0,1.0,1.0,7.444737,175.660953,260.702400,123.314500,0.001538,0.001409,0.001664,0.000000,1.0,1.0,9.068188,0.000000,3.348500,0.000256,1.0,1.0,1.0,0.0
4,2015-12-22 16:32:00,0.000000,123.383193,1.0,1.0,1.0,251.922600,8.310445,313.572593,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.564185,0.000256,136.323223,1.0,1.0,1.0,1.0,1.0,1.0,0.000,169.597557,0.000000,132.162273,1.0,1.0,1.0,1.0,1.0,7.443733,175.698533,260.747270,123.338877,0.001538,0.001409,0.001664,0.000034,1.0,1.0,9.068188,0.000000,3.313787,0.000256,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16495,2015-12-28 09:57:30,2.556886,520.460163,2.0,2.0,1.0,262.117613,8.392004,328.743903,2.441782,2.0,1.0,1.0,2.0,1.0,2.0,1.0,19.697378,2.210396,948.864637,1.0,2.0,1.0,1.0,1.0,2.0,148.808,155.853213,1.716268,932.355810,1.0,2.0,1.0,1.0,2.0,7.876250,144.885900,264.842373,12.266090,1.726062,1.282836,0.735563,0.307807,2.0,1.0,251.054733,1.674516,189.845550,0.000128,1.0,1.0,1.0,0.0
16496,2015-12-28 09:58:00,2.544246,519.901473,2.0,2.0,1.0,261.994780,8.395080,328.728520,2.442059,2.0,1.0,1.0,2.0,1.0,2.0,1.0,19.694819,2.210802,950.720453,1.0,2.0,1.0,1.0,1.0,2.0,148.808,156.020690,1.717562,934.082277,1.0,2.0,1.0,1.0,2.0,7.877019,144.926100,264.855180,12.180211,1.728326,1.287610,0.735397,0.306763,2.0,1.0,251.252893,1.667574,189.996673,0.000128,1.0,1.0,1.0,0.0
16497,2015-12-28 09:58:30,2.546317,521.215133,2.0,2.0,1.0,261.984100,8.392432,328.685000,2.442764,2.0,1.0,1.0,2.0,1.0,2.0,1.0,19.651174,2.209111,952.979500,1.0,2.0,1.0,1.0,1.0,2.0,148.808,156.165107,1.717417,938.164577,1.0,2.0,1.0,1.0,2.0,7.876314,144.916727,264.778260,12.090490,1.728143,1.287204,0.735781,0.307876,2.0,1.0,251.173293,1.681991,189.921897,0.000128,1.0,1.0,1.0,0.0
16498,2015-12-28 09:59:00,2.540616,522.947487,2.0,2.0,1.0,262.005433,8.390680,328.567070,2.442059,2.0,1.0,1.0,2.0,1.0,2.0,1.0,19.687561,2.210276,953.821950,1.0,2.0,1.0,1.0,1.0,2.0,148.808,155.938637,1.718258,940.933093,1.0,2.0,1.0,1.0,2.0,7.876058,145.100373,264.630830,12.105872,1.728356,1.281841,0.735269,0.307171,2.0,1.0,251.176520,1.704418,189.868497,0.000128,1.0,1.0,1.0,0.0


In [3]:
df_down = pd.read_csv('/DownsampledNormal.csv')
df_down['Timestamp'] = pd.to_datetime(df_down['Timestamp'])
df_down = df_down.set_index(df_down['Timestamp'])
df_down = df_down.drop(columns=['Timestamp'])
df_train = df_down.drop(columns=['Normal/Attack'])

df_down2 = pd.read_csv('/DownsampledAttack.csv')
df_down2['Timestamp'] = pd.to_datetime(df_down2['Timestamp'])
df_down2 = df_down2.set_index(df_down2['Timestamp'])
df_down2 = df_down2.drop(columns=['Timestamp'])
df_down.shape, df_down2.shape,

((16500, 52), (14999, 52))

In [4]:
ytest = df_down2['Normal/Attack']

In [5]:
ytest.value_counts()

0    13144
1     1855
Name: Normal/Attack, dtype: int64

In [7]:
54621/(495000+449919), 1855/(16500+14999)

(0.05780495471040375, 0.05889075843677577)

In [59]:
df_down = pd.read_csv('/DownsampledNormal.csv')
df_down = df_down.set_index(df_down['Timestamp'])
df_down = df_down.drop(columns=['Timestamp'])
df_train = df_down.drop(columns=['Normal/Attack'])

In [16]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold, KFold
from sklearn import linear_model, metrics, preprocessing
from math import sqrt, floor
from datetime import datetime, timedelta
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input, LSTM,Dense,RepeatVector, TimeDistributed , LeakyReLU
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, IsolationForest
from sklearn.metrics import r2_score, make_scorer, accuracy_score, mean_squared_error, mean_absolute_error
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
import tensorflow as tf
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
import pandas as pd
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)
import time
from sklearn.pipeline import make_pipeline, Pipeline
import math
from sklearn.decomposition import PCA
from numpy import array
from keras.utils.vis_utils import plot_model
import keras
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input,Conv2D, Conv1D, Conv1DTranspose,MaxPooling1D,AveragePooling1D,  UpSampling1D
import json
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from keras import optimizers, Sequential
from sklearn.ensemble import IsolationForest
from statistics import mean
from tcn import TCN
from sklearn.metrics import f1_score, accuracy_score, precision_score,recall_score

train_path = '/train/'
test_path = '/test/'

def encoder(input_dim, encoding_dim, layer1, layer2):
    x = Dense(layer1, activation = 'relu')(input_dim)
    x = Dense(layer2, activation = 'relu')(x)
    x = Dense(encoding_dim, activation = 'relu')(x)
    return x

def decoder(encoding_dim, decod_dim, layer1, layer2):
    x = Dense(layer2, activation = 'relu')(encoding_dim)
    x = Dense(layer1, activation = 'relu')(x)
    x = Dense(decod_dim, activation = 'relu')(x)
    return x

def create_autoencoder_model2(encoding_dim = 4, input_dim = 51, decod_dim=51,layer1=40, layer2=30):
    input_dim = Input(shape = (input_dim, ))
    encoder_out = encoder(input_dim, encoding_dim, layer1, layer2)
    decoder_out = decoder(encoder_out, decod_dim, layer1, layer2)
    autoencoder = Model(inputs = input_dim, outputs = decoder_out)
    autoencoder.compile(optimizer = 'adam', loss = 'mse')
    return autoencoder

import ast
df_hyp = pd.read_csv('./AE_HYP.csv')
def getHyperParams(file):
    param_dict = ast.literal_eval(df_hyp.loc[df_hyp['file'] == file]['Hyperparameters'].values[0])
    print(param_dict)
    return param_dict['model__batch_size'], param_dict['model__encoding_dim'], param_dict['model__epochs'],param_dict['model__layer1'], param_dict['model__layer2'] 

callback1 = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=1)

df_down = pd.read_csv('/DownsampledNormal.csv')
df_down['Timestamp'] = pd.to_datetime(df_down['Timestamp'])
df_down = df_down.set_index(df_down['Timestamp'])
df_down = df_down.drop(columns=['Timestamp'])
ytrain_df = df_down[['Normal/Attack']]
df_train = df_down.drop(columns=['Normal/Attack'])

df_down2 = pd.read_csv('/DownsampledAttack.csv')
df_down2['Timestamp'] = pd.to_datetime(df_down2['Timestamp'])
df_down2 = df_down2.set_index(df_down2['Timestamp'])
df_down2 = df_down2.drop(columns=['Timestamp'])
ytest = df_down2['Normal/Attack']
df_test = df_down2.drop(columns=['Normal/Attack'])
format = '%Y-%m-%d %H:%M:%S'
end_date = datetime.strptime(str(df_train.tail(1).index[0]), format)
lst=[]
source_dict_results = {}
source_dict_results_train = {}
for i in [1,2,4,7]:
    start_date = end_date-timedelta(days=i)
    ytrain = ytrain_df[(ytrain_df.index > start_date) & (ytrain_df.index < end_date)]
    x_train = df_train[(df_train.index > start_date) & (df_train.index < end_date)]
    
    scalar = MinMaxScaler()
    scalar.fit(x_train)
    xtrain = scalar.transform(x_train)
    xtest = scalar.transform(df_test)
    print(x_train.shape)
    model__batch_size,model__encoding_dim, model__epochs, model__layer1, model__layer2 = getHyperParams(i)                                               
    model  = create_autoencoder_model2(layer1=model__layer1, layer2=model__layer2, encoding_dim=model__encoding_dim)
    model.fit(xtrain, xtrain, epochs=model__epochs, batch_size=model__batch_size, verbose=False,validation_split=0.1, callbacks=callback1)
    test_enc = model.predict(xtest)
    train_enc = model.predict(xtrain)
    test_mses = np.mean(np.power(xtest- test_enc, 2), axis=1)
    train_mses = np.mean(np.power(xtrain- train_enc, 2), axis=1)
    test_maes = np.mean(xtest- test_enc, axis=1)
    train_maes = np.mean(xtrain- train_enc, axis=1)
    mse_threshold = np.max(train_mses)
    mae_threshold = np.max(train_maes)
    ypred_mse = np.where(test_mses>mse_threshold,1,0)
    ypred_mae = np.where(test_maes>mae_threshold,1,0)
    lst.append(str(i)+' days'+', F1: '+str(f1_score(y_true=ytest, y_pred=ypred_mse))+', Acc: '+str(accuracy_score(y_true=ytest, y_pred=ypred_mse))+', Prec: '+str(precision_score(y_true=ytest, y_pred=ypred_mse))+', Rec: '+str(recall_score(y_true=ytest, y_pred=ypred_mse)))
    print(lst)
    df = pd.DataFrame({'Model':['AE'],'days':[str(i)+'D'],'F1':[str(f1_score(y_true=ytest, y_pred=ypred_mse))], 'Acc':[str(accuracy_score(y_true=ytest, y_pred=ypred_mse))], 'Prec':[str(precision_score(y_true=ytest, y_pred=ypred_mse))], 'Rec':[str(recall_score(y_true=ytest, y_pred=ypred_mse))]})
    df.to_csv('./SWaTResults/metrics.csv', mode = 'a', header=False)
    test_df = pd.DataFrame(test_mses)
    test_df['mse_scores'] = test_df[0]
    test_df['ytrue'] = ytest
    test_df['mae_scores'] = test_maes
    test_df['ypred_mse'] = ypred_mse
    test_df['ypred_mae'] = ypred_mae
    test_df['Datetime'] = ytest.index
    test_df = test_df[['Datetime','ytrue','ypred_mse','ypred_mae','mse_scores', 'mae_scores']]
    source_dict_results[i] = test_df
    
    train_df = pd.DataFrame(train_mses)
    train_df['mse_scores'] = train_df[0]
    train_df['ytrue'] = ytrain.values
    train_df['mae_scores'] = train_maes
    train_df['Datetime'] = ytrain.index
    train_df = train_df[['Datetime','ytrue','mse_scores', 'mae_scores']]
    source_dict_results_train[i] = train_df
import pickle
with open(test_path+'test_STL_AE.pkl', 'wb') as f:
    pickle.dump(source_dict_results, f) 
with open(train_path+'train_STL_AE.pkl', 'wb') as f:
    pickle.dump(source_dict_results_train, f) 

(2879, 51)
{'model__batch_size': 256, 'model__encoding_dim': 30, 'model__epochs': 200, 'model__layer1': 50, 'model__layer2': 30}
['1 days, F1: 0.25064414543372465, Acc: 0.30195346356423763, Prec: 0.14450771643145993, Rec: 0.9439353099730459']
(5759, 51)
{'model__batch_size': 128, 'model__encoding_dim': 20, 'model__epochs': 200, 'model__layer1': 30, 'model__layer2': 40}
['1 days, F1: 0.25064414543372465, Acc: 0.30195346356423763, Prec: 0.14450771643145993, Rec: 0.9439353099730459', '2 days, F1: 0.2579150579150579, Acc: 0.359290619374625, Prec: 0.1505182514646237, Rec: 0.9002695417789758']
(11519, 51)
{'model__batch_size': 128, 'model__encoding_dim': 30, 'model__epochs': 200, 'model__layer1': 50, 'model__layer2': 30}
['1 days, F1: 0.25064414543372465, Acc: 0.30195346356423763, Prec: 0.14450771643145993, Rec: 0.9439353099730459', '2 days, F1: 0.2579150579150579, Acc: 0.359290619374625, Prec: 0.1505182514646237, Rec: 0.9002695417789758', '4 days, F1: 0.2573099415204678, Acc: 0.348023201546

In [3]:
lst

['1 days, F1: 0.25689027561102445, Acc: 0.3330888725915061, Prec: 0.14897466827503017, Rec: 0.9320754716981132',
 '2 days, F1: 0.2565041580834669, Acc: 0.3502900193346223, Prec: 0.14939566299324564, Rec: 0.9061994609164421',
 '4 days, F1: 0.262972060178078, Acc: 0.3598239882658844, Prec: 0.1533160297144903, Rec: 0.9234501347708894',
 '7 days, F1: 0.2749000595389981, Acc: 0.43162877525168347, Prec: 0.16319935366592608, Rec: 0.8711590296495957']

In [49]:
lst

['2 days, F1: 0.2562733582487987, Acc: 0.34988999266617776, Prec: 0.14925373134328357, Rec: 0.9056603773584906',
 '4 days, F1: 0.2590819929384072, Acc: 0.3704246949796653, Prec: 0.15160697887970614, Rec: 0.8900269541778976',
 '7 days, F1: 0.2745560138035519, Acc: 0.4253616907793853, Prec: 0.16267703969678834, Rec: 0.879245283018868']

In [14]:
train_df

,Datetime,ytrue,mse_scores,mae_scores
0,2015-12-22 16:30:00,0.0,0.009331,0.012296
1,2015-12-22 16:30:30,0.0,0.009343,0.012220
2,2015-12-22 16:31:00,0.0,0.009328,0.012368
3,2015-12-22 16:31:30,0.0,0.009318,0.012379
4,2015-12-22 16:32:00,0.0,0.009316,0.012364
...,...,...,...,...
16494,2015-12-28 09:57:00,0.0,0.001792,0.008101
16495,2015-12-28 09:57:30,0.0,0.002014,0.008253
16496,2015-12-28 09:58:00,0.0,0.002012,0.008307
16497,2015-12-28 09:58:30,0.0,0.002041,0.008630


In [1]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold, KFold
from sklearn import linear_model, metrics, preprocessing
from math import sqrt, floor
from datetime import datetime, timedelta
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input, LSTM,Dense,RepeatVector, TimeDistributed , LeakyReLU
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, IsolationForest
from sklearn.metrics import r2_score, make_scorer, accuracy_score, mean_squared_error, mean_absolute_error
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
import tensorflow as tf
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
import pandas as pd
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)
import time
from sklearn.pipeline import make_pipeline, Pipeline
import math
from sklearn.decomposition import PCA
from numpy import array
from keras.utils.vis_utils import plot_model
import keras
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input,Conv2D, Conv1D, Conv1DTranspose,MaxPooling1D,AveragePooling1D,  UpSampling1D
import json
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from keras import optimizers, Sequential
from sklearn.ensemble import IsolationForest
from statistics import mean
from tcn import TCN


def encoder(input_dim, encoding_dim, layer1, layer2):
    x = Dense(layer1, activation = 'relu')(input_dim)
    x = Dense(layer2, activation = 'relu')(x)
    x = Dense(encoding_dim, activation = 'relu')(x)
    return x

def decoder(encoding_dim, decod_dim, layer1, layer2):
    x = Dense(layer2, activation = 'relu')(encoding_dim)
    x = Dense(layer1, activation = 'relu')(x)
    x = Dense(decod_dim, activation = 'relu')(x)
    return x

def create_autoencoder_model2(encoding_dim = 4, input_dim = 51, decod_dim=51,layer1=40, layer2=30):
    input_dim = Input(shape = (input_dim, ))
    encoder_out = encoder(input_dim, encoding_dim, layer1, layer2)
    decoder_out = decoder(encoder_out, decod_dim, layer1, layer2)
    autoencoder = Model(inputs = input_dim, outputs = decoder_out)
    autoencoder.compile(optimizer = 'adam', loss = 'mse')
    return autoencoder

callback1 = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=1)

df_down = pd.read_csv('/mnt/stud/home/vpulagura/experiment/OpenDatasets/SWaT/data/DownsampledNormal.csv')
df_down['Timestamp'] = pd.to_datetime(df_down['Timestamp'])
df_down = df_down.set_index(df_down['Timestamp'])
df_down = df_down.drop(columns=['Timestamp'])
df_train = df_down.drop(columns=['Normal/Attack'])
format = '%Y-%m-%d %H:%M:%S'
end_date = datetime.strptime(str(df_train.tail(1).index[0]), format)
for i in [1]:#,2,4,7]:
    start_date = end_date-timedelta(days=i)
    x_train = df_train[(df_train.index > start_date) & (df_train.index < end_date)]
    scalar = MinMaxScaler()
    scalar.fit(x_train)
    x_train = scalar.transform(x_train)
    
    regressor_keras = KerasRegressor(create_autoencoder_model2, verbose=0)
    pipeline_keras = Pipeline([('model', regressor_keras)])
    param_grid_keras ={
            'model__epochs': [100, 200],
            'model__batch_size': [256, 128],
            'model__layer1':[50 ,40,30],
            'model__layer2':[40,30, 20],
            'model__encoding_dim':[30, 20, 10, 5]
            }


    gds_keras = GridSearchCV(
            pipeline_keras,
            param_grid=param_grid_keras,
            scoring='neg_mean_squared_error',
            cv=2, 
            refit=True,
            verbose=2, n_jobs = 1
        )
    gds_keras.fit(x_train, x_train)

    best_params = gds_keras.best_params_
    df = pd.DataFrame({'file':[i],'model':['AE'], 'Hyperparameters':[gds_keras.best_params_]})
    print(df)
    if i==2:
        df.to_csv('/mnt/stud/home/vpulagura/experiment/OpenDatasets/SWaT/AE_HYP.csv')
    else:
        df.to_csv('/mnt/stud/home/vpulagura/experiment/OpenDatasets/SWaT/AE_HYP.csv', mode='a', header=None)

2023-06-22 13:01:31.479353: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-22 13:01:31.479448: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Fitting 2 folds for each of 144 candidates, totalling 288 fits


2023-06-22 13:02:56.603697: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-22 13:02:56.604334: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-06-22 13:02:56.604760: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-06-22 13:02:56.605303: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2023-06-22 13:02:56.605953: W tensorflow/stream_executor/platform/default/dso_loader.cc:64

[CV] END model__batch_size=256, model__encoding_dim=30, model__epochs=100, model__layer1=50, model__layer2=40; total time=   9.5s
[CV] END model__batch_size=256, model__encoding_dim=30, model__epochs=100, model__layer1=50, model__layer2=40; total time=   3.6s
[CV] END model__batch_size=256, model__encoding_dim=30, model__epochs=100, model__layer1=50, model__layer2=30; total time=   2.7s
[CV] END model__batch_size=256, model__encoding_dim=30, model__epochs=100, model__layer1=50, model__layer2=30; total time=   2.8s
[CV] END model__batch_size=256, model__encoding_dim=30, model__epochs=100, model__layer1=50, model__layer2=20; total time=   3.0s
[CV] END model__batch_size=256, model__encoding_dim=30, model__epochs=100, model__layer1=50, model__layer2=20; total time=   3.0s
[CV] END model__batch_size=256, model__encoding_dim=30, model__epochs=100, model__layer1=40, model__layer2=40; total time=   2.8s
[CV] END model__batch_size=256, model__encoding_dim=30, model__epochs=100, model__layer1=4